# Usuarios y títulos

In [1]:
import ast
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

## Préstamos

In [2]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 20.1 s, sys: 845 ms, total: 20.9 s
Wall time: 20.9 s


In [3]:
prestamos_df.shape

(3054360, 20)

In [4]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


## Perfiles 2016

In [5]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
    'Rango_edad': 'category',
}

perfiles_df = pd.read_csv(
    'usuarios_2016.csv', header=0, dtype=dtypes)
# perfiles_df = perfiles_df.fillna('')

CPU times: user 136 ms, sys: 0 ns, total: 136 ms
Wall time: 135 ms


In [6]:
perfiles_df.shape

(50272, 12)

In [7]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad,Rango_edad
0,88180782,BLAA,OTRO,chapinero,bogota,CIN,universidad,OTRO,OTRO,NaN,-1.0,OTRO
1,88183608,BLAA,sra.,OTRO,bogota,CEM,pensionado,OTRO,OTRO,1957-07-30 00:00:00,59.0,50-64
2,88181732,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,NaN,-1.0,OTRO
3,88182347,ARM,sr.,OTRO,OTRO,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53.0,50-64
4,88181284,BLAA,sr.,soledad,bogota,CEM,OTRO,OTRO,OTRO,NaN,-1.0,OTRO


## Mezcla

### Merge

In [8]:
mezcla_df = pd.merge(
    prestamos_df,
    perfiles_df,
    left_on='COD_BARRAS_LECTOR',
    right_on='CODBAR',
)

In [9]:
len(mezcla_df)

773509

In [10]:
mezcla_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,...,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad,Rango_edad
0,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,...,sr.,OTRO,armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
1,Genealogía de una bruja,"Lacombe, Benjamin, 1982-",ARM,2266448,843,Infantil,2009,sp,Empleado,Técnico,...,sr.,OTRO,armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
2,La independencia de Colombia : así fue,"Vasco, Irene, 1952-",ARM,2199410,986,Infantil,2009,ck,Empleado,Técnico,...,sr.,OTRO,armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
3,Maravillas del mundo,"Steele, Philip.",ARM,2091333,910,Infantil,2007,mx,Empleado,Técnico,...,sr.,OTRO,armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
4,No te gustaría ser un adivino maya,"Matthews, Rupert.",ARM,2097624,972,Infantil,2008,ck,Empleado,Técnico,...,sr.,OTRO,armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49


### Ciudades y departamentos

#### Ciudades

In [11]:
mezcla_df.Ciudad.nunique()

80

In [12]:
mezcla_df.Ciudad.unique()

array(['armenia', 'pereira', 'circasia', 'bogota', 'cali', 'bucaramanga',
       'OTRO', 'cucuta', 'manizales', 'medellin', 'calarca', 'neiva',
       'dosquebradas', 'buenaventura', 'monteria', 'villavicencio',
       'tunja', 'floridablanca', 'valledupar', 'riohacha', 'ibague',
       'popayan', 'soledad', 'girardot', 'leticia', 'mosquera',
       'santa marta', 'envigado', 'soacha', 'cartagena', 'zipaquira',
       'sincelejo', 'cajica', 'florencia', 'barranquilla', 'facatativa',
       'sibate', 'chia', 'villamaria', 'funza', 'san andres', 'madrid',
       'piedecuesta', 'sopo', 'ipiales', 'pasca', 'tabio', 'cartago',
       'honda', 'acacias', 'paipa', 'bello', 'cota', 'mariquita',
       'flandes', 'quibdo', 'chiquinquira', 'la calera', 'rivera', 'tenjo',
       'duitama', 'santa rosa', 'fusagasuga', 'palermo', 'sogamoso',
       'ricaurte', 'espinal', 'chinchina', 'timbio', 'motavita', 'oicata',
       'combita', 'turbaco', 'guaduas', 'tocancipa', 'piendamo', 'tello',
       'sa

In [32]:
ciudad_ciudad_dict = {
    'OTRO': 'OTRO',
    'acacias': 'Acacías',
#     'aipe': 'Aipe',
    'armenia': 'Armenia',
    'barranquilla': 'Barranquilla',
#     'baraya': 'Baraya',
    'bello': 'Bello',
    'bogota': 'Bogotá',
    'bucaramanga': 'Bucaramanga',
    'buenaventura': 'Buenaventura',
    'cajica': 'Cajicá',
    'calarca': 'Calarcá',
    'cali': 'Cali',
    'cartagena': 'Cartagena',
    'cartago': 'Cartago',
    'chia': 'Chía',
    'chinchina': 'Chinchiná',
    'chiquinquira': 'Chiquinquirá',
    'circasia': 'Circasia',
    'combita': 'Cómbita',
    'cota': 'Cota',
    'cucuta': 'Cúcuta',
    'dosquebradas': 'Dosquebradas',
    'duitama': 'Duitama',
    'envigado': 'Envigado',
    'espinal': 'Espinal',
    'facatativa': 'Facatativá',
    'flandes': 'Flandes',
    'florencia': 'Florencia',
    'floridablanca': 'Floridablanca',
    'funza': 'Funza',
    'fusagasuga': 'Fusagasugá',
    'girardot': 'Girardot',
    'guaduas': 'Guaduas',
    'honda': 'Honda',
    'ibague': 'Ibagué',
    'ipiales': 'Ipiales',
    'la calera': 'La Calera',
    'leticia': 'Leticia',
    'madrid': 'Madrid',
    'manizales': 'Manizales',
    'mariquita': 'Mariquita',
    'medellin': 'Medellín',
    'monteria': 'Montería',
    'mosquera': 'Mosquera',
    'motavita': 'Motavita',
    'neiva': 'Neiva',
    'oicata': 'Oicatá',
    'paipa': 'Paipa',
    'palermo': 'Palermo',
    'pasca': 'Pasca',
    'pasto': 'San Juan de Pasto', #
    'pereira': 'Pereira',
    'piedecuesta': 'Piedecuesta',
    'piendamo': 'Piendamó',
    'popayan': 'Popayán',
    'quibdo': 'Quibdó',
    'ricaurte': 'Ricaurte',
    'riohacha': 'Riohacha',
    'rivera': 'Rivera',
    'salento': 'Salento',
    'san andres': 'San Andrés',
    'santa marta': 'Santa Marta',
    'santa rosa': 'Santa Rosa',
    'sibate': 'Sibaté',
    'sincelejo': 'Sincelejo',
    'soacha': 'Soacha',
    'sogamoso': 'Sogamoso',
    'soledad': 'Soledad',
    'sopo': 'Sopó',
    'tabio': 'Tabio',
    'tello': 'Tello',
    'tenjo': 'Tenjo',
    'timbio': 'Timbío',
    'tocaima': 'Tocaima',
    'tocancipa': 'Tocancipá',
    'tunja': 'Tunja',
    'turbaco': 'Turbaco',
    'valledupar': 'Valledupar',
    'villamaria': 'Villamaría',
    'villavicencio': 'Villavicencio',
#     'villavieja': 'Villavieja',
    'yaguara': 'Yaguará',
    'zipaquira': 'Zipaquirá',
}

In [14]:
%%time

def process_ciudad(x):
    return ciudad_ciudad_dict[x['Ciudad']]

mezcla_df['Ciudad'] = mezcla_df.apply(process_ciudad, axis=1)

CPU times: user 1min 46s, sys: 155 ms, total: 1min 46s
Wall time: 1min 46s


In [15]:
mezcla_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,...,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad,Rango_edad
0,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,...,sr.,OTRO,Armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
1,Genealogía de una bruja,"Lacombe, Benjamin, 1982-",ARM,2266448,843,Infantil,2009,sp,Empleado,Técnico,...,sr.,OTRO,Armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
2,La independencia de Colombia : así fue,"Vasco, Irene, 1952-",ARM,2199410,986,Infantil,2009,ck,Empleado,Técnico,...,sr.,OTRO,Armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
3,Maravillas del mundo,"Steele, Philip.",ARM,2091333,910,Infantil,2007,mx,Empleado,Técnico,...,sr.,OTRO,Armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49
4,No te gustaría ser un adivino maya,"Matthews, Rupert.",ARM,2097624,972,Infantil,2008,ck,Empleado,Técnico,...,sr.,OTRO,Armenia,CEM,empleado,técnico,adm. de empresas,1973-09-11 00:00:00,43.0,40-49


#### Departamentos

In [42]:
dtypes = {
    'Ciudad': str,
    'Población': np.int32,
    'Departamento': str,
}

ciudades_df = pd.read_csv(
    'ciudades_colombia.csv',
    sep='\t',
    header=-1,
    names=['Wiki index', 'Ciudad', 'Población', 'Departamento'],
    usecols=['Ciudad', 'Población', 'Departamento'],
    dtype=dtypes,
    thousands='.',
)

ciudades_departamentos_sr = pd.Series(list(ciudades_df['Departamento']), index=ciudades_df['Ciudad'])
ciudades_departamentos_sr['OTRO'] = 'OTRO'

In [43]:
ciudades_departamentos_sr.head()

Ciudad
Bogotá             Cundinamarca
Medellín              Antioquia
Cali            Valle del Cauca
Barranquilla          Atlántico
Cartagena               Bolívar
dtype: object

In [49]:
sucursal_ciudad_dict = {
    'ARM': 'armenia',
    'BAR': 'barranquilla',
    'BLAA': 'bogota',
    'BUE': 'buenaventura',
    'BUC': 'bucaramanga',
    'CAL': 'cali',
    'CAR': 'cartagena',
    'CASAGO': 'bogota',
    'CATMAN': 'bogota',
    'CUC': 'cucuta',
    'FLO': 'florencia',
    'GIR': 'girardot',
    'HON': 'honda',
    'IBA': 'ibague',
    'IEC': 'bogota',
    'IPI': 'ipiales',
    'LET': 'leticia',
    'MAN': 'manizales',
    'MED': 'medellin',
    'MON': 'monteria',
    'NEI': 'neiva',
    'PAS': 'pasto',
    'PER': 'pereira',
    'POP': 'popayan',
    'QUI': 'quibdo',
    'RIO': 'riohacha',
    'SAN': 'san andres',
    'SIN': 'sincelejo',
    'STM': 'santa marta',
    'TUN': 'tunja',
    'VAL': 'valledupar',
    'VIL': 'villavicencio',
}

In [50]:
sucursal_departamento_dict = {}
for s, c in sucursal_ciudad_dict.items():
    c = ciudad_ciudad_dict[c]
    dept = ciudades_departamentos_sr[c]
    if isinstance(dept, str):
        sucursal_departamento_dict[s] = dept
    else:
        print(dept)

Florencia    Caquetá
Florencia      Cauca
dtype: object
Armenia      Quindío
Armenia    Antioquia
dtype: object
San Andrés    Archipiélago de San Andrés
San Andrés                     Santander
dtype: object


In [51]:
sucursal_departamento_dict['FLO'] = 'Caquetá'
sucursal_departamento_dict['ARM'] = 'Quindío'
sucursal_departamento_dict['SAN'] = 'Archipiélago de San Andrés'

In [59]:
%%time

a = b = c = 0

def process_departamento(x):
    ciudad = x['Ciudad']
    dept = ciudades_departamentos_sr[ciudad]
    if isinstance(dept, str):
        r = dept
    else:
        r = sucursal_departamento_dict[x['SUCURSAL_DEL_LECTOR']]
        if r not in dept.values:
            qqq.add((ciudad, tuple(dept.values), r))
    return 

mezcla_df['Departamento'] = mezcla_df.apply(process_departamento, axis=1)

CPU times: user 3min 28s, sys: 200 ms, total: 3min 28s
Wall time: 3min 27s


In [68]:
# %%time

qqq=set()
a=b=c=d=e=0

def process_departamento(x):
    ciudad = x['Ciudad']
    dept = ciudades_departamentos_sr[ciudad]
    if isinstance(dept, str):
        r = dept
        global a
        a+=1
        if r != sucursal_departamento_dict[x['SUCURSAL_DEL_LECTOR']]:
            global b
            b+=1
    else:
        r = sucursal_departamento_dict[x['SUCURSAL_DEL_LECTOR']]
        global c
        c+=1
        if r not in dept.values:
            qqq.add((ciudad, tuple(dept.values), r))
            global d
            d+=1
    global e
    e+=1
    return 

mezcla_df['Departamento'] = mezcla_df.apply(process_departamento, axis=1)

KeyError: ('', 'occurred at index 245127')

In [ ]:
print(a)
print(b)
print(c)
print(d)
print(e)

In [63]:
ccc # / 773509

1006

In [64]:
qqq

{('Armenia', ('Quindío', 'Antioquia'), 'Amazonas'),
 ('Armenia', ('Quindío', 'Antioquia'), 'Caldas'),
 ('Armenia', ('Quindío', 'Antioquia'), 'Cundinamarca'),
 ('Armenia', ('Quindío', 'Antioquia'), 'Córdoba'),
 ('Armenia', ('Quindío', 'Antioquia'), 'Nariño'),
 ('Armenia', ('Quindío', 'Antioquia'), 'Risaralda'),
 ('Armenia', ('Quindío', 'Antioquia'), 'Tolima'),
 ('Florencia', ('Caquetá', 'Cauca'), 'Cundinamarca'),
 ('Florencia', ('Caquetá', 'Cauca'), 'Huila'),
 ('Florencia', ('Caquetá', 'Cauca'), 'Risaralda'),
 ('Mosquera', ('Cundinamarca', 'Nariño'), 'Santander'),
 ('San Andrés', ('Archipiélago de San Andrés', 'Santander'), 'Antioquia'),
 ('San Andrés', ('Archipiélago de San Andrés', 'Santander'), 'Bolívar'),
 ('San Andrés', ('Archipiélago de San Andrés', 'Santander'), 'Cundinamarca'),
 ('Santa Rosa', ('Bolívar', 'Cauca'), 'Caldas'),
 ('Santa Rosa', ('Bolívar', 'Cauca'), 'Risaralda')}

In [8]:
prestamos_cols = [
    'TITULO',
    'AUTOR',
#     'SUCURSAL',
#     'COD_BARRAS_LIBRO',
#     'DEWEY',
#     'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB',
    'PAIS_EJEMPLAR',
#     'OCUPACION',
#     'NIVEL_EDUCACION',
#     'AREA_ESTUDIO',
#     'SUCURSAL_PRESTAMO',
#     'SUCURSAL_DEVOLUCION',
#     'SUCURSAL_DEL_LECTOR',
    'COD_BARRAS_LECTOR',
#     'CATEGORÍA',
#     'FECHA_PRESTAMO',
#     'FECHA_REAL_DEVOLUCION',
#     'DATE_PRESTAMO',
#     'DATE_DEVOLUCION',
]

In [9]:
perfiles_cols = [
    'CODBAR',
#     'Sucursal',
    'Tratamiento',
#     'Barrio',
    'Ciudad',
#     'CATEG',
#     'Ocupación1',
#     'Nivel Estudio1',
#     'Área de Estudio1',
#     'Fech. Nacim',
#     'Edad',
    'Rango_edad',
]

### Ciudades y departamentos

In [19]:
mezcla_df.head()

,TITULO,AUTOR,FECHA_PUB,PAIS_EJEMPLAR,CODBAR,Tratamiento,Ciudad,Rango_edad
0,Johannes Gutenberg,"Borràs Perelló, Lluís.",2010,sp,88007424,sr.,armenia,40-49
1,Genealogía de una bruja,"Lacombe, Benjamin, 1982-",2009,sp,88007424,sr.,armenia,40-49
2,La independencia de Colombia : así fue,"Vasco, Irene, 1952-",2009,ck,88007424,sr.,armenia,40-49
3,Maravillas del mundo,"Steele, Philip.",2007,mx,88007424,sr.,armenia,40-49
4,No te gustaría ser un adivino maya,"Matthews, Rupert.",2008,ck,88007424,sr.,armenia,40-49


In [ ]:

mezcla_df.drop('COD_BARRAS_LECTOR', axis=1, inplace=True)

### Valores

In [ ]:
mezcla_df.Tratamiento.value_counts()

In [ ]:
mezcla_df.Rango_edad.value_counts()

In [ ]:
mezcla_df.Departamento.value_counts()

In [18]:
In [38]: left = pd.DataFrame({'keyl': ['K0', 'K1', 'K2', 'K3'],
   ....:                      'A': ['A0', 'A1', 'A2', 'A3'],
   ....:                      'B': ['B0', 'B1', 'B2', 'B3']})
   ....: 

In [39]: right = pd.DataFrame({'keyr': ['K4', 'K1', 'K2', 'K5'],
   ....:                       'C': ['C0', 'C1', 'C2', 'C3'],
   ....:                       'D': ['D0', 'D1', 'D2', 'D3']})
   ....: 

In [40]: result = pd.merge(left, right, left_on='keyl', right_on='keyr')

In [19]:
left

,A,B,keyl
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K2
3,A3,B3,K3


In [20]:
right

,C,D,keyr
0,C0,D0,K4
1,C1,D1,K1
2,C2,D2,K2
3,C3,D3,K5


In [21]:
result

,A,B,keyl,C,D,keyr
0,A1,B1,K1,C1,D1,K1
1,A2,B2,K2,C2,D2,K2
